In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys

sys.path.append('../')  # 返回notebook的上一级目录
# sys.path.append('E:\GitHub\QA-abstract-and-reasoning')  # 效果同上

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
np.set_printoptions(suppress=True)
from utils.loader import *
from utils.config import *

from gensim.models.word2vec import LineSentence, Word2Vec
import tensorflow as tf
# from model_layer import seq2seq_model
import time

# 加载数据

In [3]:
train_x,train_y,test_x = load_dataset()  # 数据集
vocab_index,index_vocab = load_vocab(VOCAB_INDEX_PAD)  # vocab
embedding_matrix = np.loadtxt(EMBEDDING_MATRIX_PAD)  # 预训练层

In [6]:
# 输入的长度  train_X.shape -> (82871, 261)
input_length = train_x.shape[1]
# 输出的长度  train_Y.shape -> (82871, 34)
output_sequence_length = train_y.shape[1]
# 词表大小
vocab_size=len(vocab_index)
print("输入长度：{}\n输出长度：{}\n词表大小：{}".format(input_length, output_sequence_length, vocab_size))

输入长度：260
输出长度：33
词表大小：32566


## 基本参数设置

In [7]:
# 取部分数据进行训练
sample_num=640
train_X=train_x[:sample_num]
train_Y=train_y[:sample_num]

In [8]:
# 训练集的长度
BUFFER_SIZE = len(train_X)

# 输入的长度
max_length_inp=train_X.shape[1]
# 输出的长度
max_length_targ=train_Y.shape[1]

BATCH_SIZE = 64

# 训练一轮需要迭代多少步
steps_per_epoch = len(train_X)//BATCH_SIZE

# 词向量维度
embedding_dim = 300

# 隐藏层单元数
units = 1024

# 词表大小
vocab_size = len(vocab_index)

# 构建训练集
dataset = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

## 构建Encoder

In [9]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim ,embedding_matrix , enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units # whats this
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,weights=[embedding_matrix],trainable=False)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        # (一批数据大小, 隐层的维数)
        return tf.zeros((self.batch_sz, self.enc_units))

In [10]:
encoder = Encoder(vocab_size, embedding_dim,embedding_matrix, units, BATCH_SIZE)


Encoder output shape: (batch size, sequence length, units) (64, 260, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [18]:
example_input_batch = tf.cast(train_x[:64], dtype=tf.int32)
print(example_input_batch.shape)
example_input_batch

(64, 260)


<tf.Tensor: id=140, shape=(64, 260), dtype=int32, numpy=
array([[32562,   403,   985, ..., 32565, 32565, 32565],
       [32562,   816, 26474, ..., 32565, 32565, 32565],
       [32562,  1445,    81, ...,    31,     2, 32564],
       ...,
       [32562,  4387, 29078, ..., 32565, 32565, 32565],
       [32562,   167,     7, ..., 32565, 32565, 32565],
       [32562,  2152,   952, ..., 32565, 32565, 32565]])>

In [21]:
sample_hidden = encoder.initialize_hidden_state()
sample_hidden

<tf.Tensor: id=146, shape=(64, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [23]:
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 260, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


<img src="picture/gru.png" width = "50%" height = "50%" />